# Sentiment Analysis of Covid-19 Related Tweets based on Geographic Location and State Policies

#### Authors: Steve Diamond [(GitHub)](ttps://github.com/ssdiam2000), Markell Jones-Francis [(GitHub)](https://git.generalassemb.ly/markelljones-francis), and Julia Kelman [(GitHub)](https://git.generalassemb.ly/julia-kelman/)

## Problem Statement

The COVID-19 response has been largely regional and state-based in nature. Some states have enacted strictly enforced stay-at-home policies, while others have provided guidelines. Now, some states are rapidly moving to reopen while others are following scientific guidelines and moving towards reopening very deliberately.   
While data concerning the number of Covid-19 cases and deaths has been reported for each state and analyzed based on the states' policies on social distancing, little information is known about the relationship between those features and individuals' sentiments about the pandemic.  
To get a full picture of the U.S.'s response to the coronavirus,  there is a need to look at the sentiment analysis of social media posts across geographic regions and compare them to both the local policies on social distancing and the occurrences of the pandemic in those areas.  
In order to fill this information gap, we will map the sentiment score based on tweets related to Covid-19 posted between April 28th and May 6th, 2020 accross U.S states.     
In order to gain a deeper understanding of the relationship between sentiment and location, we will compare the sentiment of residents in New York and Texas, two large states on opposite ends of the political spectrum which have acted very differently in terms of the crises.  
More specifically, we will build and interpret a Linear Regression model using Covid-19 occurences and state policy information to predict the sentiment score of NY and Texas residents. 

## Executive Summary

## Table of Contents

- [Loading Data](#Loading-Data)
    - [Library Imports](#Library-Imports)  
    - [Data Imports](#Data-Imports)
    - [Data Dictionary](#Data-Dictionary)
- [Data Cleaning](#Data-Cleaning) 
- [Exploratory Data Analysis](#EDA)
- [Mapping](#Mapping)
- [Model Preparation](#Model-Preparation)
- [Modeling](#Modeling)  
    - [Baseline Model](#Baseline-Model)  
    - [Linear Regression](#Linear-Regression)
    - [OLS](#OLS)
- [Model Selection](#Model-Selection)
- [Model Evaluation](#Model-Evaluation)
    - [Coefficients Interpretation](#Coefficients-Interpretation)
- [Conclusion](#Conclusion)
- [References](#References)

## Loading Data 

### Library Imports

### Data Imports

### Data Dictionary 

|**Feature Name**|**Description**|
|:---|:---|
|all_gatherings_prohibited|Indicator variable - 1 if all gatherings prohibited in county|
|businesses_reopening|Indicator variable - 1 if state has reopened business|
|cases|Number of Covid 19 cases in user's county|
|city|Twtter user's city name|
|county_fips|Standard ID number for user county location|
|county_name|Twtter user county name|
|date||
|deaths|Number of Covid 19 deaths in user's county|
|density|Population density of city|
|hashtags|Hashtags|
|lat|Latitude location of city|
|likes|Number of likes|
|lng|Longitude location of city|
|population|Population of city|
|ranking|?|
|replies|Number of replies|
|retweets|Number of retweets|
|screen_name|Twitter user screen name|
|sentiment_score|Sentiment score from TextBlob|
|shelter_in_place|Indicator variable - 1 if shelter_in_place order in effect for county|
|state_id|Two-digit state ID for user location|
|state_name|Twtter user state location|
|state|Twtter user's state name|
|state_stay_at_home|Indicator variable - 1 if statewide stay-at-home order in place|
|temporary_hospitals|Indicator variable - 1 if state has created temporary hospitals|
|tests_per_thousand|Number of tests per thousand in state conducted by 4/28|
|test_positives_rate|Percent of the tests in state conducted by 4/28 that were positive|
|text|Tweet text|
|traveler_quarantines|Indicator variable - 1 if state has had traveler quarantines|
|tweet_id|Tweet ID number - used to identify tweet information|
|tweet_timestamp|Timestamp of Tweet|
|username|Twitter user username|

## Data Cleaning

## EDA

## Mapping

## Model Preparation

## Modeling

### Baseline Model

### Linear Regression

### OLS

## Model Selection

## Model Evaluation

### Coefficients Interpretation

## Conclusion

## References